# SCENIC

STEP 1: Gene regulatory network inference, and generation of co-expression modules
Phase Ia: GRN inference using the GRNBoost2 algorithm
For this step the CLI version of SCENIC is used. This step can be deployed on an High Performance Computing system. We use the counts matrix (without log transformation or further processing) from the loom file we wrote earlier. Output: List of adjacencies between a TF and its targets stored in ADJACENCIES_FNAME.

In [ ]:
%%bash
source activate scverse-py38
pyscenic grn data/adrenal_Parse_10x_integrated_RNA_subset.loom data/25_tfs.txt -o result/25_tf_subset_adj.tsv --num_workers 8

Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.

2022-12-08 09:50:03,907 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2022-12-08 09:50:05,734 - pyscenic.cli.pyscenic - INFO - Inferring regulatory networks.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork fro

STEP 2-3: Regulon prediction aka cisTarget from CLI
For this step the CLI version of SCENIC is used. This step can be deployed on an High Performance Computing system.

Output: List of adjacencies between a TF and its targets stored in MOTIFS_FNAME.

locations for ranking databases, and motif annotations:

In [155]:
%%bash
source activate single_cell_py
pyscenic ctx adj.tsv \
    ../hg38__refseq-r80__10kb_up_and_down_tss.mc9nr.feather \
    --annotations_fname ../motifs-v9-nr.hgnc-m0.001-o0.0.tbl \
    --expression_mtx_fname pbmc10k_filtered_scenic.loom \
    --output reg.csv \
    --mask_dropouts \
    --num_workers 16

[########################################] | 100% Completed | 44.6s



2022-02-16 16:56:07,341 - pyscenic.cli.pyscenic - INFO - Creating modules.

2022-02-16 16:56:07,354 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2022-02-16 16:56:12,765 - pyscenic.utils - INFO - Calculating Pearson correlations.

2022-02-16 16:56:12,769 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [True].

2022-02-16 16:56:13,096 - pyscenic.utils - INFO - Creating modules.

2022-02-16 16:56:14,248 - pyscenic.cli.pyscenic - IN

STEP 4: Cellular enrichment (aka AUCell) from CLI
It is important to check that most cells have a substantial fraction of expressed/detected genes in the calculation of the AUC. The following histogram gives an idea of the distribution and allows selection of an appropriate threshold. In this plot, a few thresholds are highlighted, with the number of genes selected shown in red text and the corresponding percentile in parentheses). See the relevant section in the R tutorial for more information.

By using the default setting for --auc_threshold of 0.05, we see that 1192 genes are selected for the rankings based on the plot below.

In [156]:
%%bash
source activate single_cell_py
pyscenic aucell \
    pbmc10k_filtered_scenic.loom \
    reg.csv \
    --output pyscenic_output.loom \
    --num_workers 20

Create regulons from a dataframe of enriched features.
Additional columns saved: []



2022-02-16 16:59:46,516 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2022-02-16 16:59:52,708 - pyscenic.cli.pyscenic - INFO - Loading gene signatures.

2022-02-16 16:59:52,778 - pyscenic.cli.pyscenic - INFO - Calculating cellular enrichment.

2022-02-16 17:00:02,294 - pyscenic.cli.pyscenic - INFO - Writing results to file.


---

# Scratch